<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/elemi10-7506-Trabajo-Practico-2/Tp2_7506.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tp-2 Org de datos( FIUBA)


    
      
      
      
***

***
### Importacion de librerias

In [ ]:
#IMPORT FILES FROM DRIVE INTO GOOGLE-COLAB:

#STEP-1: Import Libraries

# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
#STEP-2: Autheticate E-Mail ID

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#STEP-3: Get File from Drive using file-ID

#2.1 Get the file
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

downloaded1 = drive.CreateFile({'id':'17pAgG9oJRK1bAFWRKkp96__zicG6yUmy'}) # replace the id with id of file you want to access
downloaded1.GetContentFile('test.csv') 

downloaded2 = drive.CreateFile({'id':'1u8v51BT7FZggIRD-eo0dQno--0wlxIhA'}) # replace the id with id of file you want to access
downloaded2.GetContentFile('sample_submission.csv') 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download("popular")
from sklearn import model_selection
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import HashingVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator
from sklearn.preprocessing import FunctionTransformer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

### Archivos necesarios

In [ ]:
train=pd.read_csv(r"train.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
test=pd.read_csv(r"test.csv")
test.tail()

,id,keyword,location,text
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...
3262,10875,NaN,NaN,#CityofCalgary has activated its Municipal Eme...


In [ ]:
sample_submission=pd.read_csv(r"sample_submission.csv")
sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


## Generacion de algunas variables y nuevos DF

In [ ]:
train_text=train.text
train_target=train.target
test_text=test.text

##### Datasets que con nulos rellenados como 'none'
****
* Despues probar rellenando con 'nokeyword','nolocation'

In [ ]:
train_na=train.fillna(value='none')
test_na=test.fillna(value='none')
train_na['keyword+text']=train_na.text+''+train_na.keyword
test_na['keyword+text']=test_na.text+''+test_na.keyword
train_na['Caracteres']=train_na.text.str.len()
test_na['Caracteres']=test_na.text.str.len()
train_na.tail()

,id,keyword,location,text,target,keyword+text,Caracteres
7608,10869,none,none,Two giant cranes holding a bridge collapse int...,1,Two giant cranes holding a bridge collapse int...,83
7609,10870,none,none,@aria_ahrary @TheTawniest The out of control w...,1,@aria_ahrary @TheTawniest The out of control w...,125
7610,10871,none,none,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,65
7611,10872,none,none,Police investigating after an e-bike collided ...,1,Police investigating after an e-bike collided ...,137
7612,10873,none,none,The Latest: More Homes Razed by Northern Calif...,1,The Latest: More Homes Razed by Northern Calif...,94


### Clasificadores

In [ ]:
# Establecemos a la regresion logistica como clasificador
# Arbol como clasificador
cls=LogisticRegression()
clf = DecisionTreeClassifier(random_state=0)
gbc = GradientBoostingClassifier(random_state=0)
rfc=RandomForestClassifier(random_state=0)


# *MODELOS*
 
  
   
    
     
      

######   **Modelo 0**: Simple

In [ ]:
#Vectorizer, simple 
vectorizer=CountVectorizer()
vectorizer.fit(train_text)
X_train=vectorizer.transform(train_text)
X_test=vectorizer.transform(test_text)
X_train

<7613x21637 sparse matrix of type '<class 'numpy.int64'>'
	with 111497 stored elements in Compressed Sparse Row format>

###### *Modelo 1*: Agregando stopwords y eliminando caracteres tiles y ese tipo de cosas

In [ ]:
#Vectorizer1, agregando stopwords y distintos n_grams
stopwords=stopwords.words('english')
vectorizer1=CountVectorizer(strip_accents='ascii',stop_words=stopwords,ngram_range=(1,5),max_df=0.95)
vectorizer1.fit(train_text)
X_train1=vectorizer1.transform(train_text)
X_test1=vectorizer1.transform(test_text)
X_train1

<7613x232480 sparse matrix of type '<class 'numpy.int64'>'
	with 344172 stored elements in Compressed Sparse Row format>

###### Modelo 2: Uso de TF-IDF 

In [ ]:
#Vectorizer2, usando TF-IDF
vectorizer2=TfidfVectorizer()
vectorizer2.fit(train_text)
X_train2=vectorizer2.transform(train_text)
X_test2=vectorizer2.transform(test_text)
X_train2

<7613x21637 sparse matrix of type '<class 'numpy.float64'>'
	with 111497 stored elements in Compressed Sparse Row format>

###### Modelo 3: Uso de TF-IDF,sacando acentos y analizando varios n_grams

In [ ]:
# Vectorizer3, usando TF-iDF, sacando acentos y agreando n_grams
max_n3=15
vectorizer3=TfidfVectorizer(strip_accents='ascii',ngram_range=(1,max_n3),)
vectorizer3.fit(train_text)
X_train3=vectorizer3.transform(train_text)
X_test3=vectorizer3.transform(test_text)
X_train3

<7613x769185 sparse matrix of type '<class 'numpy.float64'>'
	with 1004105 stored elements in Compressed Sparse Row format>

###### Modelo 4: Inclusion de Lemmatizador, NO incluye uso de TF-IDF

In [ ]:
# Vectorizer4
# Creamos un lemmatizador 
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

vectorizer4=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer4.fit(train_text)
X_train4=vectorizer4.transform(train_text)
X_test4=vectorizer4.transform(test_text)
X_train4

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<7613x22003 sparse matrix of type '<class 'numpy.int64'>'
	with 128166 stored elements in Compressed Sparse Row format>

###### Modelo 5( mejores resultados):  
- Uso de columna keyword + text
- Lemmatizador

In [ ]:
train_text_y_keyword=train_na['keyword+text']
test_text_y_keyword=test_na['keyword+text']

In [ ]:
# Utilizo keyword en mi analisis
vectorizer5=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer5.fit(train_text_y_keyword)
X_train5=vectorizer5.transform(train_text_y_keyword)
X_test5=vectorizer5.transform(test_text_y_keyword)
X_train5

###### Modelo 6: Utilizo keyword en mi analisis y TF-IDF

In [ ]:
# Utilizo keyword en mi analisis y TF-IDF
vectorizer6=TfidfVectorizer(tokenizer=LemmaTokenizer())
vectorizer6.fit(train_text_y_keyword)
X_train6=vectorizer6.transform(train_text_y_keyword)
X_test6=vectorizer6.transform(test_text_y_keyword)
X_train6

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<7613x24172 sparse matrix of type '<class 'numpy.float64'>'
	with 130285 stored elements in Compressed Sparse Row format>

###### Modelo 7: Inclusion de columna location

In [ ]:
# Agregamos columna location 
train_text_keyword_location=train_na.text+''+train_na.keyword+''+train_na.location
test_text_keyword_location=test_na.text+''+test_na.keyword+''+test_na.location

In [ ]:
vectorizer7=CountVectorizer(tokenizer=LemmaTokenizer())
vectorizer7.fit(train_text_keyword_location)
X_train7=vectorizer7.transform(train_text_keyword_location)
X_test7=vectorizer7.transform(test_text_keyword_location)
X_train7

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<7613x26489 sparse matrix of type '<class 'numpy.int64'>'
	with 138938 stored elements in Compressed Sparse Row format>

###### Modelo 8: Utilizamos funcion train_test_split.....
- No tiene ningun tipo de mejora al modelo, es solo para probar, se puede utlizar para calcular el Score en lugar de hacer sub
- Despues se generaliza para todo el set de entrenamiento y se predice set de test

In [ ]:
X=train_na['keyword+text']
y=train_na['target']
X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.20, random_state=0)

In [ ]:
vectorizer8=CountVectorizer(strip_accents='ascii',tokenizer=LemmaTokenizer())
vectorizer8.fit(X_train_)
X_train8=vectorizer8.transform(X_train_)
X_test8=vectorizer8.transform(X_test_)
X_train8

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<6090x20596 sparse matrix of type '<class 'numpy.int64'>'
	with 104553 stored elements in Compressed Sparse Row format>

In [ ]:
## Calculamos Score con este metodo, sin hacer cross validation
cls.fit(X_train8,y_train_)
prediction0=cls.predict(X_test8)
print(f1_score(y_test_,prediction0))

0.7618243243243243


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
### Progabmos el score con algunos de los clasificadores
gbc.fit(X_train8,y_train_)
cls.fit(X_train8,y_train_)
prediction0=cls.predict(X_test8)
prediction1=gbc.predict(X_test8)
print('RL: {}'.format(f1_score(y_test_,prediction0)))
print('GBC: {}'.format(f1_score(y_test_,prediction1)))

RL: 0.7618243243243243
GBC: 0.6611570247933884


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## Clasificador propio
- Lemmatizador
- One-hot
- Cuenta cantidad de caracteres

In [ ]:
class Lemmatizer(BaseEstimator):
    def __init__(self):
        self.l = WordNetLemmatizer()
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = map(lambda r:  ' '.join([self.l.lemmatize(i.lower()) for i in r.split()]), x)
        x = np.array(list(x))
        return x

In [ ]:
def length_text(x):
    return np.array(x.str.len()).reshape(-1,1)

In [ ]:
lm=Lemmatizer()
vec=CountVectorizer()
lr=LogisticRegression()
clasificador=Pipeline([
    ('features',FeatureUnion([
        ('text',Pipeline([
            ('lm',lm),
            ('vec',vec)])),
        ('lenght',Pipeline([
            ('car',FunctionTransformer(length_text,validate=False))
        ]))
    ])),
    ('lr',lr)
])


- *Cross validation del nuevo clasificador*
- *El texto es text+keyword*

In [ ]:
cross_val_score(estimator=clasificador,X=train_text_y_keyword,y=train_na.target,cv=6,scoring='f1')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


array([0.61366181, 0.59171598, 0.59357278, 0.52380952, 0.6471183 ,
       0.72643253])

* Calculamos f1 para el nuevo clasificador  (Sigue usando el df spliteado)

In [ ]:
clasificador.fit(X_train_,y_train_)
clas_predict=clasificador.predict(X_test_)
f1_score(clas_predict,y_test_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7350993377483444

* Generalizamos para todo el set de train, asi tiene mayor corpus
* Hacemos submit con nuevo clasificador
* (No tiene el mejor resultado en kaggle, seria bueno hacer un grid search)

In [ ]:
# Ahora hacemos que utilice todo el dataset, asi tiene mayor corpus
clasificador.fit(train_text_y_keyword,train_na.target)
sample_submission['target']=clasificador.predict(test_text_y_keyword)
sample_submission.to_csv('sub_9(class).csv',index=False)

In [ ]:
# Por curisidad 
print(X_train[0].todense())
print(X_train1[0].todense())
print(X_train2[0].todense())
print(X_train3[0].todense())
print(X_train4[0].todense())

[[0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]]
[[0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]]
[[0 1 0 ... 0 0 0]]


In [ ]:
# Establecemos un score, usamos f1 que es el utilizado en competencia
score=model_selection.cross_val_score(cls,X_train,train.target,cv=10,scoring='f1')
score

C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.62956522, 0.56218058, 0.51779935, 0.5152    , 0.5800317 ,
       0.6048    , 0.58983051, 0.52532833, 0.70788253, 0.72941176])

In [ ]:
score1=model_selection.cross_val_score(cls,X_train1,train.target,cv=10,scoring='f1')
score1


C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.58113208, 0.47524752, 0.46704331, 0.53214286, 0.49446494,
       0.57692308, 0.55893536, 0.42173913, 0.65964912, 0.72758037])

In [ ]:
score2=model_selection.cross_val_score(cls,X_train2,train.target,cv=10,scoring='f1')
score2

C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.64084507, 0.62239089, 0.56624319, 0.5915493 , 0.61217391,
       0.62650602, 0.61202186, 0.57685009, 0.69505963, 0.74598071])

In [ ]:
score3=model_selection.cross_val_score(cls,X_train3,train.target,cv=10,scoring='f1')
score3

C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.2915601 , 0.24415584, 0.28426396, 0.25      , 0.21705426,
       0.22942643, 0.19672131, 0.29648241, 0.31862745, 0.46052632])

In [ ]:
score4=model_selection.cross_val_score(cls,X_train4,train.target,cv=10,scoring='f1')
score4

C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.64248705, 0.53767123, 0.53269537, 0.53125   , 0.609375  ,
       0.57463884, 0.6013289 , 0.53141831, 0.72977099, 0.73669065])

In [ ]:
score4_=model_selection.cross_val_score(clf,X_train4,train.target,cv=10,scoring='f1')
score4_

array([0.52886406, 0.57189542, 0.47284345, 0.46381579, 0.54205607,
       0.52365931, 0.53924915, 0.53003534, 0.59810127, 0.65486726])

In [ ]:
score5=model_selection.cross_val_score(cls,X_train5,train_na.target,cv=6,scoring='f1')
score5

C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.65005417, 0.57838364, 0.59558117, 0.54929577, 0.69147894,
       0.74649205])

In [ ]:
score5_=model_selection.cross_val_score(clf,X_train5,train_na.target,cv=10,scoring='f1')
score5_

array([0.54243542, 0.53505007, 0.48717949, 0.48648649, 0.56835637,
       0.55054432, 0.58897638, 0.58762887, 0.66049383, 0.62208398])

In [ ]:
score5_gbc=score5_=model_selection.cross_val_score(gbc,X_train5,train_na.target,cv=10,scoring='f1')

In [ ]:
score5_gbc

array([0.56214149, 0.5754717 , 0.35135135, 0.56766917, 0.48405253,
       0.54888508, 0.46990291, 0.57509158, 0.65734266, 0.65326633])

In [ ]:
score5_rfc=model_selection.cross_val_score(rfc,X_train5,train_na.target,cv=10,scoring='f1')
score5_rfc

C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

array([0.53927813, 0.48888889, 0.42352941, 0.41509434, 0.51737452,
       0.46095238, 0.49173554, 0.33103448, 0.5300207 , 0.64631957])

In [ ]:
score6=model_selection.cross_val_score(cls,X_train6,train_na.target,cv=10,scoring='f1')
score6

C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.66181818, 0.6369863 , 0.5659051 , 0.59375   , 0.63032368,
       0.60207612, 0.64027539, 0.55491329, 0.72361809, 0.75      ])

In [ ]:
score7=model_selection.cross_val_score(cls,X_train7,train_na.target,cv=8,scoring='f1')
score7

C:\Users\jinci\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.68802228, 0.57281553, 0.59055118, 0.60947503, 0.57364341,
       0.639201  , 0.64054054, 0.73293413])

In [ ]:
score7>score5

array([ True,  True,  True,  True,  True, False,  True, False,  True,
       False])

In [ ]:
score5>score6

array([False, False, False, False, False, False, False, False, False,
       False])

In [ ]:
clf.fit(X_train5,train_na.target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [ ]:
#Ajusto para el 5to modelo
cls.fit(X_train7,train_na.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# De nuevo, trabajo con el 4to modelo
sample_submission['target']=cls.predict(X_test7)


In [ ]:
#Creo el csv con el cuarto modelo
sample_submission.to_csv('sub8.csv',index=False)